In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import DataLoader

In [2]:
x=torch.load('../chess/xData.pt')
y=torch.load('../chess/yData.pt')

In [3]:
xtest=x.view(-1,2*8*8).numpy()
ytest=y.numpy()
xtest.shape

(22637, 128)

In [4]:
dataset=np.zeros(129*22637).reshape(22637,129)
dataset[:,0]=ytest
dataset[:,1:129]=xtest

In [20]:
whitewins=dataset[np.where(dataset[:,0]==0)[0]]
black=dataset[np.where(dataset[:,0]==1)[0]]
print(whitewins.shape,black.shape)

(15342, 129) (7295, 129)


In [33]:
white=whitewins[np.arange(7295)*2]
print(white.shape,black.shape)

(7295, 129) (7295, 129)


In [42]:
w=torch.FloatTensor(white)
b=torch.FloatTensor(black)
datas=torch.cat((w,b))

In [43]:
datas.shape

torch.Size([14590, 129])

In [89]:
trainset=DataLoader(datas, batch_size=2048, shuffle=True)

In [86]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.c0=nn.Conv2d(2,2,kernel_size=(8,8),padding=4)
        self.c1=nn.Conv2d(2,2,kernel_size=(2,2))
        self.c2=nn.Conv2d(2,4,kernel_size=(1,1))
        self.c3=nn.Conv2d(4,4,kernel_size=(1,1))
        self.c4=nn.Conv2d(4,8,kernel_size=(1,1))
        self.c5=nn.Conv2d(8,16,kernel_size=(1,1))
        self.c6=nn.Conv2d(16,16,kernel_size=(1,1))
        self.c7=nn.Conv2d(16,8,kernel_size=(2,2))
        self.c8=nn.Conv2d(8,4,kernel_size=(2,2))
        self.c9=nn.Conv2d(4,2,kernel_size=(2,2))
        self.c10=nn.Conv2d(2,1,kernel_size=(2,2))
        self.l1=nn.Linear(16,1)

    def forward(self, x):
        x = F.normalize(x)
        #print(x.shape)
        x = F.normalize(F.relu(self.c0(x)))
        #print(x.shape)
        x = F.normalize(F.relu(self.c1(x)))
        #print(x.shape)
        x = F.normalize(F.relu(self.c2(x)))
        #print(x.shape)
        x = F.normalize(F.relu(self.c3(x)))
        #print(x.shape)
        x = F.normalize(F.relu(self.c4(x)))
        #print(x.shape)
        x = F.normalize(F.relu(self.c5(x)))
        #print(x.shape)
        x = F.normalize(F.relu(self.c6(x)))
        #print(x.shape)
        x = F.normalize(F.relu(self.c7(x)))
        #print(x.shape)
        x = F.normalize(F.relu(self.c8(x)))
        #print(x.shape)
        x = F.normalize(F.relu(self.c9(x)))
        #print(x.shape)
        x = F.relu(self.c10(x))
        #print(x.shape)        
        x=x.view(x.shape[0],16)
        #print(x.shape)
        x = torch.sigmoid(self.l1(x))
        #print(x.shape)
        return x
    
    

print(Net())
model=Net().cuda()

Net(
  (c0): Conv2d(2, 2, kernel_size=(8, 8), stride=(1, 1), padding=(4, 4))
  (c1): Conv2d(2, 2, kernel_size=(2, 2), stride=(1, 1))
  (c2): Conv2d(2, 4, kernel_size=(1, 1), stride=(1, 1))
  (c3): Conv2d(4, 4, kernel_size=(1, 1), stride=(1, 1))
  (c4): Conv2d(4, 8, kernel_size=(1, 1), stride=(1, 1))
  (c5): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
  (c6): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
  (c7): Conv2d(16, 8, kernel_size=(2, 2), stride=(1, 1))
  (c8): Conv2d(8, 4, kernel_size=(2, 2), stride=(1, 1))
  (c9): Conv2d(4, 2, kernel_size=(2, 2), stride=(1, 1))
  (c10): Conv2d(2, 1, kernel_size=(2, 2), stride=(1, 1))
  (l1): Linear(in_features=16, out_features=1, bias=True)
)


In [91]:
learning_rate = 1e-1
criterion = nn.MSELoss(reduction="sum")
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=learning_rate/20)

In [92]:
num_epochs=1000
for epoch in range(num_epochs):
    outs=[]
    i=0
    loss_total=0
    for data in trainset:
        X= data[:,1:129]
        X=X.view(X.shape[0],2,8,8)
        y=data[:,0].view(X.shape[0],1)
        X = torch.autograd.Variable(X).cuda()
        # ===================forward=====================
        output = model(X)
        outs.append(output.cpu())
        #print(output.shape)
        loss = criterion(output, y.cuda())
        # ===================backward====================
        loss.backward()
        optimizer.step()
        loss_total=loss_total+float(loss)
        i=i+1
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss_total))

epoch [1/1000], loss:7295.0000
epoch [2/1000], loss:7295.0000
epoch [3/1000], loss:7295.0000
epoch [4/1000], loss:7295.0000
epoch [5/1000], loss:7295.0000
epoch [6/1000], loss:7295.0000
epoch [7/1000], loss:7295.0000
epoch [8/1000], loss:7288.9923
epoch [9/1000], loss:7295.0000
epoch [10/1000], loss:7295.0000
epoch [11/1000], loss:7295.0000
epoch [12/1000], loss:7295.0000
epoch [13/1000], loss:7295.0000
epoch [14/1000], loss:7295.0000
epoch [15/1000], loss:7295.0000
epoch [16/1000], loss:7295.0000
epoch [17/1000], loss:7295.0000
epoch [18/1000], loss:7295.0000
epoch [19/1000], loss:7295.0000
epoch [20/1000], loss:7295.0000
epoch [21/1000], loss:7295.0000
epoch [22/1000], loss:7295.0000


KeyboardInterrupt: 

In [ ]:
c=0
for i in yset:
    for j in i:
        if j==1:
            c=c+1
print(c)
        